In [1]:
import os
import numpy as np
import gensim
from gensim.models import Word2Vec

## Extracting Sentences from data.txt file

In [2]:
DATA_TEXT_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/data_txt.TXT"

In [3]:
all_sentences = []

In [4]:
opened_file = open(DATA_TEXT_PATH, "r", encoding="utf-8-sig")
content = opened_file.read()

content = content.split("\n")

c = 0
for line in content:
    try:
        sentence = line.split("|")[1]
        sentence1, sentence2 = sentence.split(":")
        sentence1 = sentence1.strip().split(" ")
        sentence2  =sentence2.strip()
        sentence2 = sentence2.replace('"', "")
        sentence2 = sentence2.replace(".", " .")
        all_sentences.append(sentence1)
        if "/" in sentence2:
            for sent in sentence2.split("/"):
                all_sentences.append(sent.split(" "))
        else:
            all_sentences.append(sentence2.split(" "))
    except:
        pass

In [5]:
len(all_sentences)

55504

In [6]:
all_sentences[:5]

[['ज्यास', 'जन्म', 'नाही', 'असा'],
 ['ईश्वर', 'अजन्मा', 'आहे'],
 ['शुभ', 'नाही', 'असा'],
 ['या', 'योगामूळे', 'कुंडलीतील', 'इतर', 'अशुभ', 'योगांचा', 'नाश', 'होतो', '.'],
 ['ज्याने', 'प्रवेश', 'केला', 'नाही', 'असा']]

In [7]:
for sentence in all_sentences:
    if "जाते" in sentence:
        print(sentence)
        break

['ती', 'दर', 'सोमवारी', 'शिवालयात', 'जाते', '.']


## Extracting Tourism Domain Sentences

In [8]:
TOURISM_FILE_PATH1 = "/content/drive/MyDrive/MARATHI_WSD_Data/2. MAR-TOURISM/MAR-TOURISM"
TOURISM_FILE_PATH2 = "/content/drive/MyDrive/MARATHI_WSD_Data/Marathi Tourism Domain/Marathi Tourism Domain"

In [9]:
all_lines = []
for i in os.listdir(TOURISM_FILE_PATH1):
    if i!=".ipynb_checkpoints":
        text_file = open(os.path.join(TOURISM_FILE_PATH1, i), "r", encoding="utf-8-sig")
        content = text_file.read()
        content = content.split("\n")
        lines = []
        for i in content:
            lines.append(i.split(" "))
        line_words = []
        for i in lines:
            words = []
            for j in i:
                if "#" in j:
                    words.append(j.split("#"))
            all_lines.append(words)
            
for i in os.listdir(TOURISM_FILE_PATH2):
    if i!=".ipynb_checkpoints":
        text_file = open(os.path.join(TOURISM_FILE_PATH2, i), "r", encoding="utf-8")
        content = text_file.read()
        content = content.split("\n")
        lines = []
        for i in content:
            lines.append(i.split(" "))
        line_words = []
        for i in lines:
            words = []
            for j in i:
                if "#" in j:
                    words.append(j.split("#"))
            all_lines.append(words)

In [10]:
all_lines_word_info = []
for line in all_lines:
    words_info = []
    if line != []:
        for word in line:
            word_info = {}
            word_info["word"] = word[0]
            if word[1] !="" and word[1][0] == "{":
                word_info["root_word"] = word[1][1:len(word[1])-1].split(",")
            else:
                word_info["root_word"] = word[1]
            word_info["pos"] = word[2]
            word_info["word_position"] = word[3]
            word_info["word_id"] = word[4]
            words_info.append(word_info)
        all_lines_word_info.append(words_info)

In [11]:
for line in all_lines_word_info:
    print(line)
    break

[{'word': '\ufeffचित्रगुप्त', 'root_word': '\ufeffचित्रगुप्त', 'pos': 'n', 'word_position': 'w1', 'word_id': '26981'}, {'word': 'मंदिर', 'root_word': 'मंदिर', 'pos': 'n', 'word_position': 'w2', 'word_id': '452'}, {'word': 'हे', 'root_word': ['हा', 'हे'], 'pos': 'u', 'word_position': 'w3', 'word_id': '0'}, {'word': 'सूर्य', 'root_word': 'सूर्य', 'pos': 'n', 'word_position': 'w4', 'word_id': '2186'}, {'word': 'देवाचे', 'root_word': 'देव', 'pos': 'n', 'word_position': 'w5', 'word_id': '2484'}, {'word': 'मंदिर', 'root_word': 'मंदिर', 'pos': 'n', 'word_position': 'w6', 'word_id': '452'}, {'word': 'आहे', 'root_word': 'आह', 'pos': 'u', 'word_position': 'w7', 'word_id': '0'}, {'word': 'तसेच', 'root_word': 'तसे', 'pos': 'r', 'word_position': 'w8', 'word_id': '10811'}, {'word': 'विश्वनाथ', 'root_word': 'विश्वनाथ', 'pos': 'n', 'word_position': 'w9', 'word_id': '19174'}, {'word': 'मंदिर', 'root_word': 'मंदिर', 'pos': 'n', 'word_position': 'w10', 'word_id': '452'}, {'word': 'हे', 'root_word': ['हा'

In [12]:
sentences = [[]]
for word in all_lines_word_info[0]:
    if type(word["root_word"]) == list:
        l_rword = len(word["root_word"])
        l_sen = len(sentences)
        temp = sentences[-1].copy()
        if l_sen < l_rword:
            for _ in range(l_rword - l_sen):
                sentences.append(sentences[0].copy())
            for index in range(l_rword):
                sentences[index].append(word["root_word"][index])
        else:
            for index in range(l_sen):
                try:
                    sentences[index].append(word["root_word"][index])
                except:
                    sentences[index].append(word["word"])
        if word["word"] not in word["root_word"]:
            sentences.append(temp)
            sentences[-1].append(word["word"])
    else:
        if word["word"] != word["root_word"]:
            sentences.append(sentences[-1].copy())
        for sentence in sentences[:-1]:
            sentence.append(word["root_word"])
        sentences[-1].append(word["word"])
t = sentences.copy()
for sentence in sentences:
    if t.count(sentence) > 1:
        while t.count(sentence)!=1:
            del t[t.index(sentence)]
for sent in t:
    print(' '.join(sent))

﻿चित्रगुप्त मंदिर हा सूर्य देव मंदिर आह तसे विश्वनाथ मंदिर हा शिव आणि ते विश्व वाहन अस नंद हा बैल आह .
﻿चित्रगुप्त मंदिर हे सूर्य देव मंदिर आह तसे विश्वनाथ मंदिर हे शीव आणि तो विश्वासू वाहन अस नंद ही बैल आह .
﻿चित्रगुप्त मंदिर हे सूर्य देवाचे मंदिर आह तसे विश्वनाथ मंदिर हे शिवाचे आणि त्याचे विश्वासू वाहन अस नंद हे बैल आह .
﻿चित्रगुप्त मंदिर हे सूर्य देवाचे मंदिर आहे तसे विश्वनाथ मंदिर हे शिवाचे आणि त्याचे विश्वासू वाहन अस नंद ह्या बैल आह .
﻿चित्रगुप्त मंदिर हे सूर्य देवाचे मंदिर आहे तसेच विश्वनाथ मंदिर हे शिवाचे आणि त्याचे विश्वासू वाहन अस नंद ह्या बैल आह .
﻿चित्रगुप्त मंदिर हे सूर्य देवाचे मंदिर आहे तसेच विश्वनाथ मंदिर हे शिवाचे आणि त्याचे विश्वासू वाहन असलेल्या नंद ह्या बैल आह .
﻿चित्रगुप्त मंदिर हे सूर्य देवाचे मंदिर आहे तसेच विश्वनाथ मंदिर हे शिवाचे आणि त्याचे विश्वासू वाहन असलेल्या नंदी ह्या बैल आह .
﻿चित्रगुप्त मंदिर हे सूर्य देवाचे मंदिर आहे तसेच विश्वनाथ मंदिर हे शिवाचे आणि त्याचे विश्वासू वाहन असलेल्या नंदी ह्या बैलाचे आह .
﻿चित्रगुप्त मंदिर हे सूर्य देवाचे मंदिर आहे तसेच विश्

In [13]:
tourism_sentences = []

for line in all_lines_word_info:
    sentences = [[]]
    for word in line:
        if type(word["root_word"]) == list:
            l_rword = len(word["root_word"])
            l_sen = len(sentences)
            temp = sentences[-1].copy()
            if l_sen < l_rword:
                for _ in range(l_rword - l_sen):
                    sentences.append(sentences[0].copy())
                for index in range(l_rword):
                    sentences[index].append(word["root_word"][index])
            else:
                for index in range(l_sen):
                    try:
                        sentences[index].append(word["root_word"][index])
                    except:
                        sentences[index].append(word["word"])
            if word["word"] not in word["root_word"]:
                sentences.append(temp)
                sentences[-1].append(word["word"])
        else:
            if word["word"] != word["root_word"]:
                sentences.append(sentences[-1].copy())
            for sentence in sentences[:-1]:
                sentence.append(word["root_word"])
            sentences[-1].append(word["word"])
    t = sentences.copy()
    for sentence in sentences:
        if t.count(sentence) > 1:
            while t.count(sentence)!=1:
                del t[t.index(sentence)]

    tourism_sentences.extend(t)

In [14]:
print(len(all_sentences))

55504


In [15]:
all_sentences = all_sentences + tourism_sentences

In [16]:
len(all_sentences)

292220

## Extracting Health Domain Sentences

In [17]:
FILES_PATH1 = "/content/drive/MyDrive/MARATHI_WSD_Data/3. MAR-HEALTH/MAR-HEALTH"
FILES_PATH2 = "/content/drive/MyDrive/MARATHI_WSD_Data/Marathi Health Domain/Marathi Health Domain"

In [18]:
all_lines = []
for i in os.listdir(FILES_PATH1):
    if i!=".ipynb_checkpoints":
        text_file = open(os.path.join(FILES_PATH1, i), "r", encoding="utf-8-sig")
        content = text_file.read()
        content = content.split("\n")
        lines = []
        for i in content:
            lines.append(i.split(" "))
        line_words = []
        for i in lines:
            words = []
            for j in i:
                if "#" in j:
                    words.append(j.split("#"))
            all_lines.append(words)
            
for i in os.listdir(FILES_PATH2):
    if i!=".ipynb_checkpoints":
        text_file = open(os.path.join(FILES_PATH2, i), "r", encoding="utf-8")
        content = text_file.read()
        content = content.split("\n")
        lines = []
        for i in content:
            lines.append(i.split(" "))
        line_words = []
        for i in lines:
            words = []
            for j in i:
                if "#" in j:
                    words.append(j.split("#"))
            all_lines.append(words)

In [19]:
all_lines_word_info = []
for line in all_lines:
    words_info = []
    if line != []:
        for word in line:
            word_info = {}
            word_info["word"] = word[0]
            if word[1] !="" and word[1][0] == "{":
                word_info["root_word"] = word[1][1:len(word[1])-1].split(",")
            else:
                word_info["root_word"] = word[1]
            word_info["pos"] = word[2]
            word_info["word_position"] = word[3]
            word_info["word_id"] = word[4]
            words_info.append(word_info)
        all_lines_word_info.append(words_info)

In [20]:
sentences = [[]]
for word in all_lines_word_info[3]:
    if type(word["root_word"]) == list:
        l_rword = len(word["root_word"])
        l_sen = len(sentences)
        temp = sentences[-1].copy()
        if l_sen < l_rword:
            for _ in range(l_rword - l_sen):
                sentences.append(sentences[0].copy())
            for index in range(l_rword):
                sentences[index].append(word["root_word"][index])
        else:
            for index in range(l_sen):
                try:
                    sentences[index].append(word["root_word"][index])
                except:
                    sentences[index].append(word["word"])
        if word["word"] not in word["root_word"]:
            sentences.append(temp)
            sentences[-1].append(word["word"])
    else:
        if word["word"] != word["root_word"]:
            sentences.append(sentences[-1].copy())
        for sentence in sentences[:-1]:
            sentence.append(word["root_word"])
        sentences[-1].append(word["word"])
t = sentences.copy()
for sentence in sentences:
    if t.count(sentence) > 1:
        while t.count(sentence)!=1:
            del t[t.index(sentence)]
for sent in t:
    print(' '.join(sent))

काही कर येणे ऊन अस , कधी नस हा मान्य आह , पण अस " काहीच कर ये नस " हा जेव्हा आवश्यक अस तेव्हा त एक पवित्र जेफर्सनिक फसवणूक अस .
काही करता ये नाह असा , कधी नस हे मान्य आह , पण असा " काहीच कर ये नस " हे जेव्हा आवश्यक अस तेव्हा ती एक पवित्र जेफर्सनिक फसवणूक अस .
काही करता येत नाही असे , कधी नस हे मान्य आह , पण असे " काहीच कर ये नस " हे जेव्हा आवश्यक अस तेव्हा ते एक पवित्र जेफर्सनिक फसवणूक अस .
काही करता येत नाही असे , कधीच नस हे मान्य आह , पण असे " काहीच कर ये नस " हे जेव्हा आवश्यक अस तेव्हा ती एक पवित्र जेफर्सनिक फसवणूक अस .
काही करता येत नाही असे , कधीच नसते हे मान्य आह , पण असे " काहीच कर ये नस " हे जेव्हा आवश्यक अस तेव्हा ती एक पवित्र जेफर्सनिक फसवणूक अस .
काही करता येत नाही असे , कधीच नसते हे मान्य आहे , पण असे " काहीच कर ये नस " हे जेव्हा आवश्यक अस तेव्हा ती एक पवित्र जेफर्सनिक फसवणूक अस .
काही करता येत नाही असे , कधीच नसते हे मान्य आहे , पण असे " काहीच करता ये नस " हे जेव्हा आवश्यक अस तेव्हा ती एक पवित्र जेफर्सनिक फसवणूक अस .
काही करता येत नाही असे , कधीच नसते हे मान्य आहे , पण असे

In [21]:
health_sentences = []

for line in all_lines_word_info:
    sentences = [[]]
    for word in line:
        if type(word["root_word"]) == list:
            l_rword = len(word["root_word"])
            l_sen = len(sentences)
            temp = sentences[-1].copy()
            if l_sen < l_rword:
                for _ in range(l_rword - l_sen):
                    sentences.append(sentences[0].copy())
                for index in range(l_rword):
                    sentences[index].append(word["root_word"][index])
            else:
                for index in range(l_sen):
                    try:
                        sentences[index].append(word["root_word"][index])
                    except:
                        sentences[index].append(word["word"])
            if word["word"] not in word["root_word"]:
                sentences.append(temp)
                sentences[-1].append(word["word"])
        else:
            if word["word"] != word["root_word"]:
                sentences.append(sentences[-1].copy())
            for sentence in sentences[:-1]:
                sentence.append(word["root_word"])
            sentences[-1].append(word["word"])
    t = sentences.copy()
    for sentence in sentences:
        if t.count(sentence) > 1:
            while t.count(sentence)!=1:
                del t[t.index(sentence)]

    health_sentences.extend(t)

In [22]:
print(len(health_sentences), len(all_sentences))

95890 292220


In [23]:
all_sentences = all_sentences + health_sentences

In [24]:
len(all_sentences)

388110

## Extracting News Domain Sentences

In [25]:
FILE_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Marathi News Domain/new_text_marathi.TXT"

In [26]:
opened_file = open(FILE_PATH, "r", encoding="utf-8-sig")
content = opened_file.read()

In [27]:
sentences_with_pos = []
for i in content.split("\n"):
    if len(i)!=0 and i[0] != "<":
        sentences_with_pos.append(i)

In [28]:
news_sentences = []
for sentence in sentences_with_pos:
    sent = []
    for word in sentence.split(" "):
        x = word.split("_")
        if len(x)!=1:
            sent.append(word.split("_")[0])
    news_sentences.append(sent)

In [29]:
len(news_sentences)

957

In [30]:
all_sentences = all_sentences + news_sentences

In [31]:
print(len(all_sentences))

389067


In [54]:
DATA_TEXT_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/data_txt.TXT"

opened_file = open(DATA_TEXT_PATH, "r", encoding="utf-8-sig")
content = opened_file.read()

content = content.split("\n")

true_pairs = []
for line in content:
    try:
        sentence = line.split("|")[0].strip()
        words = sentence.split(" ")[3]
        if ":" in words:
            words = words.split(":")
            true_pairs.append(words)
    except:
        pass

In [55]:
print(true_pairs[:5])

[['अशुभ', 'अमंगळ'], ['पुण्यभूमी', 'पवित्रभूमी', 'पुण्यस्थान', 'पवित्रस्थान', 'पावनस्थान'], ['शिवालय', 'शिवमंदिर'], ['आलेला', 'आगत'], ['जन्मलेला', 'जलमलेला', 'जल्मलेला']]


In [56]:
for pair in true_pairs:
    if "जाते" in pair:
        print(pair)
        break

['जाते', 'घट्टी', 'चक्की']


In [34]:
pairs = true_pairs[:200]
len(pairs)

200

In [35]:
from scipy.spatial.distance import cosine

def score(pairs, model):
    score = 0
    count = 0
    for pair in pairs:
        try:
            dist = cosine(model[pair[0]], model[pair[1]])
            # dist = model.wv.similarity(pair[0], pair[1])
            score += dist
            count += 1
        except:
            pass
    return score / count

## Training a Word2vec Model on all the extracted Sentences

In [36]:
word2vec_model1 = Word2Vec(all_sentences, min_count=1, workers=3, sg = 0, size=200)

In [37]:
# word2vec_model2 = Word2Vec(all_sentences, min_count=1, workers=3, sg = 1, size=100)

In [38]:
word2vec_model1.similar_by_word("उत्पादित")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('प्रदर्शित', 0.5822898149490356),
 ('कुंचन', 0.5771068334579468),
 ('फॅशन', 0.5691759586334229),
 ('नांबियार', 0.5564556121826172),
 ('बेलगुंठाला', 0.5511257648468018),
 ('विक्री', 0.5371506214141846),
 ('सुशोभीत', 0.5349873900413513),
 ('प्रेरित', 0.5289438962936401),
 ('सोहळ्याने', 0.5196452140808105),
 ('हल्ल्यांच्या', 0.5171300172805786)]

## Word Sense Disambiguation

In [39]:
DATA_TEXT_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/data_txt.TXT"
IDX_NOUN_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/idxnoun_txt.TXT"
IDX_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/index_txt.TXT"

In [40]:
opened_file = open(IDX_NOUN_PATH, "r", encoding="utf-8-sig")
idx_noun_content = opened_file.read()

noun_content = idx_noun_content.split("\n")

nouns = []
for noun in noun_content:
    text = noun.split(" ")
    try:
        skip1 = int(text[2])
        skip2 = skip1+1
        nouns.append((text[0], text[2+skip2+1:]))
    except:
        pass

In [41]:
nouns[:5]

[('-तरंग', ['00005942']),
 ('-धारा', ['00027382']),
 ('-सारखा', ['00026718']),
 ('1000000', ['00031077']),
 ('10000000', ['00015838'])]

In [42]:
for noun in nouns:
    if noun[0] == "गिनीपिग":
        print(noun)
        break

('गिनीपिग', ['00020220'])


In [49]:
for noun in nouns:
    if noun[0] == "जाते":
        print(noun)
        break

('जाते', ['00012325'])


In [44]:
import difflib

opened_file = open(DATA_TEXT_PATH, "r", encoding="utf-8-sig")
data_content = opened_file.read()

data_content = data_content.split("\n")

all_syns = []
for syn in data_content:
    try:
        ids, sentence = syn.split(" | ")
        x = ids.split(" ")
        all_syns.append((x[0], x[3].split(":"), sentence))
    except:
        pass


nouns_data = []
for noun in nouns:
    for syn_id in noun[1]:
        for syn in all_syns:
            if syn_id == syn[0]:
                if ":" in syn[2]:
                    sentence = syn[2].split(":")[1:]
                elif "::" in syn[2]:
                    sentence = syn[2].split("::")[1:]
                sentence = ' '.join(sentence)
                sentence = sentence.replace('"',"")
                sentence = sentence.replace('.',"")
                sentence = sentence.split(" ")
                flag = False
                for synonym in syn[1]:
                    match = difflib.get_close_matches(synonym, sentence)
                    if len(match)!=0:
                        flag = True
                        sentence[sentence.index(match[0])] = noun[0]
                        break
                if flag:
                    nouns_data.append((' '.join(sentence), noun[0], syn_id))

In [45]:
for syn in all_syns:
    if syn[0] == "00020220":
        print(syn)
        break

('00020220', ['गिनीपिग'], 'शेपुट नसलेला उंदरासारखा एक सस्तन प्राणी:"गिनीपिगचे मांस लोक खातात."')


In [57]:
for syn in all_syns:
    if "जाते" in syn[1]:
        print(syn)
        break

('00012325', ['जाते', 'घट्टी', 'चक्की'], 'दोन बसके वर्तुळाकार दगड एकमेकांवर घासून मधल्या पदार्थाचा चुराडा करतील अशी योजना ज्यात असते ते दळाण्याचे साधन"आई सकाळी जाते घेऊन दळायला बसायची"')


In [46]:
for noun in nouns_data:
    if noun[1] == "गिनीपिग":
        print(noun)
        break

('गिनीपिग मांस लोक खातात', 'गिनीपिग', '00020220')


In [58]:
nouns_data_words = []
for noun in nouns_data:
    nouns_data_words.append(noun[1])
len(nouns_data), len(nouns_data_words)

(34237, 34237)

In [61]:
new_words_list = []
for pair in true_pairs:
    for word in pair:
        if word not in nouns_data_words:
            t = pair.copy()
            t.remove(word)
            for temp_word in t:
                if temp_word in nouns_data_words:
                    for t_noun in nouns_data:
                        if t_noun[1] == temp_word:
                            ans = list(t_noun)
                            ans[0] = ans[0].replace(temp_word, word)
                            ans[1] = word
                            new_words_list.append(tuple(ans))

In [62]:
len(new_words_list)

3988

In [63]:
for noun in new_words_list:
    if noun[1] == "जाते":
        print(noun)
        break

('जाते तिला चालायला त्रास होतो आहे', 'जाते', '00012325')


In [64]:
all_nouns_data = nouns_data + new_words_list
len(all_nouns_data)

38225

In [65]:
for noun in all_nouns_data:
    if noun[1] == "जाते":
        print(noun)
        break

('जाते तिला चालायला त्रास होतो आहे', 'जाते', '00012325')


In [66]:
noun_sent_data = {}
for i in all_nouns_data:
    try:
        if len(i[0].split()) != 1:
            if "::" in i[0]:
                meaning, sentence = i[0].split("::")
            elif ":" in i[0]:
                meaning, sentence = i[0].split(":")
            else:
                meaning, sentence = i[0].split(".")
            sentence = sentence[1:len(sentence)-1]
    except:
        if '"' not in i[0]:
            sentence = i[0]
        else:
            start = i[0].index('"')
            end = len(i[0])
            sentence = i[0][start+1:end-1]
    s = sentence.split("/")
    if i[1] not in noun_sent_data:
        noun_sent_data[i[1]] = {i[2]:[]}
        for sent in s:
            if i[1] in sent:
                sent = sent.replace('"', "")
                noun_sent_data[i[1]][i[2]].append(sent)
    else:
        noun_sent_data[i[1]][i[2]] = []
        for sent in s:
            if i[1] in sent:
                sent = sent.replace('"', "")
                noun_sent_data[i[1]][i[2]].append(sent)
    if len(noun_sent_data[i[1]]) == 0:
        del noun_sent_data[i[1]]
    if noun_sent_data[i[1]][i[2]] == []:
        del noun_sent_data[i[1]][i[2]]

In [67]:
noun_sent_data["गिनीपिग"]

{'00020220': ['गिनीपिग मांस लोक खातात']}

In [68]:
noun_sent_data["जाते"]

{'00012325': ['जाते जाते तुटला'], '00012326': ['जाते पट्टा तुटला']}

In [69]:
noun_sent_data["अंग"]

{'00001954': ['शरीरातील प्रत्येक अंग महत्त्वाचे असते'],
 '00001957': ['या वस्तूचा प्रत्येक अंग स्वदेशी आहे '],
 '00001961': ['आपले अंग पाच महाभूतांपासून बनलेले आहे'],
 '00003388': ['क्रिकेटाच्या खेळात सचिनचे अंग जगजाहीर आहे ',
  ' त्याला गायनाचेही अंग आहे'],
 '00003966': ['माझी उजवी अंग दुखते आहे'],
 '00010208': ['ह्या कामात मोठ्या भावाचा अंग आहे'],
 '00037159': ['दुर्योधनाने कर्णाला अंग दिला']}

In [70]:
noun_vectors = {}

for noun in noun_sent_data:
    noun_vectors[noun] = {}

    for nid in noun_sent_data[noun]:
        for sentence in noun_sent_data[noun][nid]:
            sense_vector = 0
            count = 0
            tokens = sentence.split(" ")

            noun_count = tokens.count(noun)

            if noun_count > 1:
                position = 1

                for i in range(1, noun_count + 1):
                    count = 0
                    sense_vector = 0
                    for word in tokens:
                        try:
                            if word != noun:
                                sense_vector += word2vec_model1[word]
                                count+=1
                            else:
                                if position != i:
                                    sense_vector += word2vec_model1[word]
                                    count += 1
                                position += 1
                        except:
                            pass

                    # print(len(sense_vector))
                    if count != 0:
                        if nid not in noun_vectors[noun]:
                            noun_vectors[noun][nid] = []
                            noun_vectors[noun][nid].append(sense_vector / count)
                        else:
                            noun_vectors[noun][nid].append(sense_vector / count)

            else:
                for word in tokens:
                    try:
                        if noun != word:
                            sense_vector += word2vec_model1[word]
                            count+=1
                    except:
                        pass
                # print(len(sense_vector))
                if count != 0:
                    if nid not in noun_vectors[noun]:
                        noun_vectors[noun][nid] = []
                        noun_vectors[noun][nid].append(sense_vector / count)
                    else:
                        noun_vectors[noun][nid].append(sense_vector / count)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [72]:
import math
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def getSense(sentence, noun, noun_position = 1, noun_vectors = noun_vectors):
    sentence = sentence.strip().replace(".", " .").split(" ")
    # print(sentence)

    sense_vector = 0
    count = 0
    position = 1
    for word in sentence:

        if noun!=word:
            sense_vector += word2vec_model1[word]
            count += 1
        else:
            if position != noun_position:
                sense_vector += word2vec_model1[word]
                count += 1
            position += 1

    
    if noun in noun_vectors:
        cosines = {}
        for nid in noun_vectors[noun]:
            c = []
            for vector in noun_vectors[noun][nid]:
                c.append(cosine_similarity(vector, sense_vector))
            cosines[nid] = max(c)
        
        # print(cosines)
        return max(cosines, key = cosines.get)
    else:
        return f"{noun} not in Database"

In [73]:
test_sentence = "अजमेर जिल्ह्याचे मुख्यालय अजमेर शहरात आहे."
noun = "अजमेर"

n = test_sentence.count(noun)

for i in range(n):
    print(getSense(test_sentence, noun, i+1))

00025377
00025377


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [74]:
word2vec_model1.save("word2vec_model.model")

In [75]:
loaded_model = Word2Vec.load("/content/word2vec_model.model")

In [76]:
import pickle

In [77]:
pickle.dump(noun_vectors,open("noun_vectors.pkl", "wb"))